In [1]:
import urllib.request as urllib2
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import Request, urlopen
import datetime
import bs4
from fake_useragent import UserAgent
import re 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
%matplotlib inline  
pd.set_option('display.max_columns', None)
from time import sleep
os.environ["PATH"] ="C:\dev\chromedriver;C:\dev;" + os.environ['PATH']
import warnings
warnings.filterwarnings('ignore')

To do
1. R - Get the content of the post from the post page (find preview from page with list of psots OR find the right html balise)
2. R - Improve price finding function
3. R - Find model etc
4. TGP - see if not down and do stuff

## REDDIT

In [2]:
reddit_url = "https://www.reddit.com/r/Gear4Sale/?count=25&after=t3_kqnp3f"
path="C:/dev/niche/guitar/data/"

In [74]:
def get_url(section):
    a_s=section.findAll("a")
    for a in a_s:
        if a.has_attr('href'):
            # the first link is the right one
            return a["href"]
        #TODO check link is valid

def validate_string(s):
    is_s=False
    if type(s)==str:
        if s!="":
            is_s =True
    return is_s

def find_vintage(content):
    result = ""
    # from 1985
    if "from " in content:
        split_on_from =  content.split("from ")
        if len(split_on_from)==2:
            #"guitar from 1985"
            year = split_on_from[1][:4]
            # still check that we are returning a number
            if year.isdigit():
                if int(four_char)>1900:
                    result = year
        else:
            # "guitar from the best from 1985"
            for i in range(len(split_on_from)):
                year = split_on_from[i][:4]
                # still check that we are returning a number
                if year.isdigit():
                    if int(four_char)>1900:
                        result = year
    else:
        potential_years = []
        for i in range(len(content)):
            four_char = content[i:i+4]
            if four_char.isdigit():
                if int(four_char)>1900:
                    potential_years.append(four_char)
        if len(potential_years)==1:
            # "guitar blabla 1985"
            return potential_years
        else:
            # "guitar made in 1985, bought in 2001"
            result = min(potential_years)
    return result
        # TODO check for "made in", "manufactured in" etc

def find_longest_string_of_digits(s):
    i=0
    n = len(s)
    while s[i].isdigit() or (s[i]==".") or (s[i]==","):
        i+=1
        if (i==n):
            break
    return s[:i]

def find_price(content):
    ### TODO "price", "price is", "USD", "usd"
    if "price is " in content:
        split_on_price_is = content.split("price is ")
        if len(split_on_price_is)==2:
            # price is 300,45
            # find 
            return find_longest_string_of_digits(split_on_price_is[1])
        else:
            ps = []
            # price is 45.00. price is negotiable:
            for i in range(len(split_on_price_is)):
                print(split_on_price_is[i])
                longest_digit = find_longest_string_of_digits(split_on_price_is[i])
                if longest_digit!="":
                    ps.append(longest_digit)
            if len(ps)==1:
                return ps[0]
            else:
                return ps[0]

In [127]:
response = requests.get(reddit_url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')
list_of_posts=soup.findAll("div", {"class":"rpBJOHq2PR60pnwJlUyP0"})[0].findAll("div")

df=pd.DataFrame(columns=["title","content","url"])#,"price","brand","model"])
k=0
for section in t[0:len(list_of_posts)]:
    try:
        if section.text!="":
            post_title = section.findAll("h3")[0].text
            post_content = section.find("p").text
            # get url
            post_url = get_url(section)
            # just in case the section is empty, do check title and content are legitimate strings
            if validate_string(post_title) and validate_string(post_content):
                # Check for tags:
                # only keep WTS, discard WTT and WTB
                if ("WTS" in post_title) and not ((df['title'] == post_title)).any():
                    # go to post_url
                    post_response = requests.get(post_url)
                    post_soup = bs4.BeautifulSoup(post_response.text, 'html.parser')
                    content = post_soup.text
                    vintage = find_vintage(post_title)
                    if vintage =="":
                        vintage = find_vintage(content)
                    print(post_url)
                    print(vintage)
                    print("-----")
                    # find price ?
                    # find place ?
                    # find brand
                    # find year
                    new_row = [post_title,post_content,post_url]
                    # Check that we don't already have that post in our data
                    if not ((df['title'] == post_title) & (df['content'] == post_content)).any():
                        df.loc[k]=new_row
                        k+=1

    except Exception as error:
        pass
df
        

https://www.reddit.com/r/Gear4Sale/comments/kpn8tx/wts_vester_concert_series_2_from_1985_with/
1985
1985
-----


,title,content,url
0,WTS Vester concert series 2 from 1985 with DiM...,http://imgur.com/gallery/hloDV53,https://www.reddit.com/r/Gear4Sale/comments/kp...
